In [1]:
import pathlib
import matplotlib

import mne
import mne_bids

matplotlib.use('Qt5Agg')

In [2]:
bids_root = pathlib.Path('out_data/sample_BIDS')

bids_path = mne_bids.BIDSPath(subject='01',
                              session='01',
                              task='audiovisual',
                              run='01',
                              datatype='meg',
                              root=bids_root)

raw = mne_bids.read_raw_bids(bids_path)
raw.load_data()
raw.filter(l_freq=0.1, h_freq=40)
events, event_id = mne.events_from_annotations(raw)

Opening raw data file out_data\sample_BIDS\sub-01\ses-01\meg\sub-01_ses-01_task-audiovisual_run-01_meg.fif...
    Read a total of 3 projection items:
        PCA-v1 (1 x 102)  idle
        PCA-v2 (1 x 102)  idle
        PCA-v3 (1 x 102)  idle
    Range : 25800 ... 192599 =     42.956 ...   320.670 secs
Ready.
Reading channel info from out_data\sample_BIDS\sub-01\ses-01\meg\sub-01_ses-01_task-audiovisual_run-01_channels.tsv.
Reading 0 ... 166799  =      0.000 ...   277.714 secs...


C:\Users\User\AppData\Local\Temp\ipykernel_28636\3277140186.py:10: RuntimeWarning: Did not find any events.tsv associated with sub-01_ses-01_task-audiovisual_run-01.

The search_str was "out_data\sample_BIDS\sub-01\**\meg\sub-01_ses-01*events.tsv"
  raw = mne_bids.read_raw_bids(bids_path)


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 19821 samples (33.001 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    0.6s
[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:    1.5s
[Parallel(n_jobs=1)]: Done 287 tasks      | elapsed:    2.7s


In [5]:
events = mne.find_events(raw)
print(events[:5])  # Print the first 5 events to verify the format


320 events found on stim channel STI 014
Event IDs: [ 1  2  3  4  5 32]
[[27977     0     2]
 [28345     0     3]
 [28771     0     1]
 [29219     0     4]
 [29652     0     2]]


In [6]:
event_id = {
    'Auditory/Left': 1,
    'Auditory/Right': 2,
    'Visual/Left': 3,
    'Visual/Right': 4,
    'Smiley': 5,
    'Button': 32
}

In [7]:
tmin = -0.3
tmax = 0.5
baseline = (None, 0)

epochs = mne.Epochs(raw,
                    events=events,
                    event_id=event_id,
                    tmin=tmin,
                    tmax=tmax,
                    baseline=baseline,
                    preload=True)
epochs

Not setting metadata
320 matching events found
Setting baseline interval to [-0.2996928197375818, 0.0] s
Applying baseline correction (mode: mean)
Created an SSP operator (subspace dimension = 3)
3 projection items activated
Using data from preloaded Raw for 320 events and 481 original time points ...
0 bad epochs dropped


Number of events,320
Events,Auditory/Left: 72Auditory/Right: 73Button: 16Smiley: 15Visual/Left: 73Visual/Right: 71
Time range,-0.300 – 0.499 s
Baseline,-0.300 – 0.000 s


In [8]:
epochs.plot()

Using qt as 2D backend.


Dropped 0 epochs: 
The following epochs were marked as bad and are dropped:
[]
Channels marked as bad:
['MEG 2443', 'EEG 053']


In [9]:
epochs['Auditory/Right']
epochs['Auditory']
epochs['Left']
epochs['Visual'].plot_image()

Not setting metadata
144 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
144 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
144 matching events found
No baseline correction applied
0 projection items activated
combining channels using RMS (grad channels)
combining channels using RMS (mag channels)
combining channels using GFP (eeg channels)


[<Figure size 640x480 with 3 Axes>,
 <Figure size 640x480 with 3 Axes>,
 <Figure size 640x480 with 3 Axes>]

In [10]:
epochs.save(pathlib.Path('out_data') / 'epochs_epo.fif', 
            overwrite=True)

In [11]:
evoked_auditory = epochs['Auditory'].average()
evoked_visual = epochs['Visual'].average()
evoked_auditory.plot(spatial_colors=True)
evoked_auditory.plot_topomap(ch_type='mag')
evoked_auditory.plot_joint(picks='mag')
mne.viz.plot_compare_evokeds([evoked_auditory, evoked_visual], picks='mag')

Projections have already been applied. Setting proj attribute to True.
combining channels using RMS (mag channels)
combining channels using RMS (mag channels)


[<Figure size 800x600 with 1 Axes>]

In [12]:
mne.write_evokeds(fname=pathlib.Path('out_data') / 'evokeds_ave.fif',
                  evoked=[evoked_auditory, evoked_visual])